<a href="https://colab.research.google.com/github/AngelB26/ICTAK/blob/main/Public_sentiment_Analysispynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

In [3]:
!file /content/judge-1377884607_tweet_product_company.csv


/content/judge-1377884607_tweet_product_company.csv: Non-ISO extended-ASCII text, with CR, LF line terminators


In [4]:
!iconv -f ISO-8859-1 -t UTF-8 /content/judge-1377884607_tweet_product_company.csv -o /content/judge-1377884607_tweet_product_company_utf8.csv


In [5]:
data = pd.read_csv('/content/judge-1377884607_tweet_product_company_utf8.csv')


In [6]:
data.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [8]:
# Selecting only the 'tweet_text' and 'is_there_an_emotion_directed_at_a_brand_or_product' columns for sentiment analysis

data = data[['tweet_text', 'is_there_an_emotion_directed_at_a_brand_or_product']]
data = pd.read_csv('/content/judge-1377884607_tweet_product_company_utf8.csv')


In [11]:
data.columns

Index(['tweet_text', 'emotion_in_tweet_is_directed_at',
       'is_there_an_emotion_directed_at_a_brand_or_product'],
      dtype='object')

In [13]:
# Selecting only the 'tweet_text' and 'is_there_an_emotion_directed_at_a_brand_or_product' columns for sentiment analysis

data=data[['tweet_text','is_there_an_emotion_directed_at_a_brand_or_product']]

In [14]:
# Rename columns for simplicity

data.columns = ['tweet', 'sentiment']


In [15]:
data.shape

(9093, 2)

In [16]:
pd.set_option('display.max_colwidth',None)

In [17]:
data.head()

,tweet,sentiment
0,".@wesley83 I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead! I need to upgrade. Plugin stations at #SXSW.",Negative emotion
1,"@jessedee Know about @fludapp ? Awesome iPad/iPhone app that you'll likely appreciate for its design. Also, they're giving free Ts at #SXSW",Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. They should sale them down at #SXSW.,Positive emotion
3,@sxsw I hope this year's festival isn't as crashy as this year's iPhone app. #sxsw,Negative emotion
4,"@sxtxstate great stuff on Fri #SXSW: Marissa Mayer (Google), Tim O'Reilly (tech books/conferences) &amp; Matt Mullenweg (Wordpress)",Positive emotion


In [18]:
data['sentiment'].value_counts()


No emotion toward brand or product    5389
Positive emotion                      2978
Negative emotion                       570
I can't tell                           156
Name: sentiment, dtype: int64

In [ ]:
# data preprocessing

In [19]:
data['tweet'].isnull().sum()

1

In [20]:
data['tweet'].fillna('', inplace=True)

<ipython-input-20-0b7cac3416bf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tweet'].fillna('', inplace=True)


In [21]:
# feature extraction


In [22]:
x = data['tweet']
y = data['sentiment']

In [ ]:
# encoding

In [23]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = to_categorical(y)

In [24]:
y

array([[0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       ...,
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.]], dtype=float32)

In [ ]:
# Tokenize the text data using Keras Tokenizer

In [25]:
from keras.preprocessing import text
tokenizer=text.Tokenizer()
tokenizer.fit_on_texts(list(data['tweet']))
tokenized_text=tokenizer.texts_to_sequences(data['tweet'])

In [26]:
len(tokenized_text[0])

24

In [27]:
len(tokenized_text[1])

22

In [ ]:
# Pad the tokenized_text to make all text sequences the same length (100)

In [28]:
from keras.utils import pad_sequences
x=pad_sequences(tokenized_text,maxlen=100)

In [ ]:
# train_test_split

In [29]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
# SimpleRNN

In [30]:
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding,SimpleRNN,Dropout

In [31]:
len(tokenizer.word_index)

10147

In [32]:
# Create a Sequential model
model = Sequential()

# Add an Embedding layer
model.add(Embedding(input_dim = len(tokenizer.word_index)+1, output_dim=128, input_length=100))

# Add a SimpleRNN layer with 32 units

model.add(SimpleRNN(32))
model.add(Dropout(0.5))

# Add a Dense layer with 50 units
#model.add(Dense(50,activation = 'relu'))
#model.add(Dropout(0.5))

# Add the final Dense layer with 4 units (for 4 classes)
model.add(Dense(4, activation='softmax'))


In [33]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [34]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 128)          1298944   
                                                                 
 simple_rnn (SimpleRNN)      (None, 32)                5152      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 4)                 132       
                                                                 
Total params: 1304228 (4.98 MB)
Trainable params: 1304228 (4.98 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [35]:
model.fit(x_train,y_train,epochs=10,validation_split=0.1)

Epoch 1/10
205/205 [==============================] - 14s 52ms/step - loss: 1.0286 - accuracy: 0.5501 - val_loss: 0.8596 - val_accuracy: 0.6223
Epoch 2/10
205/205 [==============================] - 10s 48ms/step - loss: 0.7166 - accuracy: 0.7290 - val_loss: 0.8536 - val_accuracy: 0.6511
Epoch 3/10
205/205 [==============================] - 10s 49ms/step - loss: 0.4783 - accuracy: 0.8390 - val_loss: 0.9626 - val_accuracy: 0.6154
Epoch 4/10
205/205 [==============================] - 9s 46ms/step - loss: 0.3376 - accuracy: 0.8923 - val_loss: 1.0114 - val_accuracy: 0.6346
Epoch 5/10
205/205 [==============================] - 9s 45ms/step - loss: 0.2603 - accuracy: 0.9140 - val_loss: 1.1193 - val_accuracy: 0.6319
Epoch 6/10
205/205 [==============================] - 10s 51ms/step - loss: 0.2310 - accuracy: 0.9218 - val_loss: 1.2018 - val_accuracy: 0.6126
Epoch 7/10
205/205 [==============================] - 10s 50ms/step - loss: 0.2066 - accuracy: 0.9300 - val_loss: 1.2013 - val_accuracy: 0

In [36]:
y_pred=model.predict(x_test)

57/57 [==============================] - 1s 8ms/step


In [37]:
accuracy = model.evaluate(x_test, y_test)[1]
print(f'Test Accuracy: {accuracy * 100:.2f}%')

57/57 [==============================] - 2s 29ms/step - loss: 1.3094 - accuracy: 0.6201
Test Accuracy: 62.01%


In [ ]:
#LSTM

In [38]:
model = Sequential()
model.add(Embedding(input_dim = len(tokenizer.word_index)+1, output_dim=128, input_length=100))
model.add(LSTM(32))
model.add(Dense(4, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train,y_train,epochs=10,validation_split=0.1)

Epoch 1/10
205/205 [==============================] - 20s 80ms/step - loss: 0.9349 - accuracy: 0.5941 - val_loss: 0.8379 - val_accuracy: 0.6236
Epoch 2/10
205/205 [==============================] - 15s 73ms/step - loss: 0.6881 - accuracy: 0.7249 - val_loss: 0.8028 - val_accuracy: 0.6621
Epoch 3/10
205/205 [==============================] - 15s 74ms/step - loss: 0.4677 - accuracy: 0.8214 - val_loss: 0.8327 - val_accuracy: 0.6607
Epoch 4/10
205/205 [==============================] - 15s 74ms/step - loss: 0.3471 - accuracy: 0.8700 - val_loss: 0.9007 - val_accuracy: 0.6374
Epoch 5/10
205/205 [==============================] - 15s 74ms/step - loss: 0.2745 - accuracy: 0.8926 - val_loss: 1.0123 - val_accuracy: 0.6538
Epoch 6/10
205/205 [==============================] - 15s 75ms/step - loss: 0.2266 - accuracy: 0.9085 - val_loss: 1.1082 - val_accuracy: 0.6470
Epoch 7/10
205/205 [==============================] - 15s 74ms/step - loss: 0.1859 - accuracy: 0.9218 - val_loss: 1.1815 - val_accuracy:

In [39]:
y_pred=model.predict(x_test)


57/57 [==============================] - 2s 21ms/step


In [40]:
accuracy = model.evaluate(x_test, y_test)[1]
print(f'Test Accuracy: {accuracy * 100:.2f}%')

57/57 [==============================] - 1s 13ms/step - loss: 1.4565 - accuracy: 0.6592
Test Accuracy: 65.92%
